In [1]:
import inspect

from getpass import getpass
from langchain import OpenAI
from langchain import OpenAI, PromptTemplate
from langchain.chains import LLMChain, ConversationChain
from langchain.chains.conversation.memory import (ConversationBufferMemory, 
                                                 ConversationSummaryMemory, 
                                                 ConversationBufferWindowMemory, 
                                                 ConversationKGMemory)
from langchain.callbacks import get_openai_callback
import tiktoken
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI

In [2]:
import os
import openai

os.environ["OPENAI_API_TYPE"] = openai.api_type = "azure"
os.environ["OPENAI_API_VERSION"] = openai.api_version = "2022-12-01" #"2023-03-15-preview"
os.environ["OPENAI_API_BASE"] = openai.api_base =  "https://idocopenaigpt.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = openai.api_key = "95776649ac7a4b048c834003fd315264"#os.getenv("AZUREOPENAI_API_KEY")

In [3]:
chat = AzureChatOpenAI(
    deployment_name="chatllm16k",
    openai_api_version="2023-03-15-preview",
    model_name="gpt-3.5-turbo",
    temperature=0,
)

C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_community\chat_models\azure_openai.py:161: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://idocopenaigpt.openai.azure.com/ to https://idocopenaigpt.openai.azure.com/openai.
  warnings.warn(
C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_community\chat_models\azure_openai.py:168: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
C:\Users\Ramshankar\anaconda3\Lib\site-packages\langchain_community\chat_models\azure_openai.py:176: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specified it is expected to be of the form https://example-resource.azure.o

Chats with the Chat-GPT model gpt-3.5-turbo are typically structured like so:

        System: You are a helpful assistant.

        User: Hi AI, how are you today?

        Assistant: I'm great thank you. How can I help you?

        User: I'd like to understand string theory.

        Assistant: 
The final `"Assistant:"` without a response is what would prompt the model to continue the comversation. In the official OpenAI `ChatCompletion` endpoint these would be passed to the model in a format like:

[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hi AI, how are you today?"},
    {"role": "assistant", "content": "I'm great thank you. How can I help you?"}
    {"role": "user", "content": "I'd like to understand string theory."}
]

In LangChain there is a slightly different format. We use three message objects like so:

In [4]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="I'd like to understand string theory.")
]

The format is very similar, we're just swapper the role of `"user"` for `HumanMessage`, and the role of `"assistant"` for `AIMessage`.

We generate the next response from the AI by passing these messages to the `ChatOpenAI` object.

In [5]:
res = chat(messages)
res

AIMessage(content="String theory is a theoretical framework in physics that aims to describe the fundamental nature of the universe. It suggests that the fundamental building blocks of the universe are not point-like particles, but tiny, vibrating strings. These strings can vibrate at different frequencies, giving rise to different particles and their properties.\n\nHere are some key points about string theory:\n\n1. Dimensions: String theory requires the existence of extra dimensions beyond the three spatial dimensions (length, width, and height) that we are familiar with. The most widely studied version of string theory, called superstring theory, suggests that there are 10 dimensions in total, with 6 of them being compactified or curled up into tiny, undetectable sizes.\n\n2. Unification: String theory attempts to unify all the fundamental forces of nature, including gravity, electromagnetism, and the strong and weak nuclear forces. It provides a framework where these forces can be 

In response we get another AI message object. We can print it more clearly like so:

In [6]:
print(res.content)

String theory is a theoretical framework in physics that aims to describe the fundamental nature of the universe. It suggests that the fundamental building blocks of the universe are not point-like particles, but tiny, vibrating strings. These strings can vibrate at different frequencies, giving rise to different particles and their properties.

Here are some key points about string theory:

1. Dimensions: String theory requires the existence of extra dimensions beyond the three spatial dimensions (length, width, and height) that we are familiar with. The most widely studied version of string theory, called superstring theory, suggests that there are 10 dimensions in total, with 6 of them being compactified or curled up into tiny, undetectable sizes.

2. Unification: String theory attempts to unify all the fundamental forces of nature, including gravity, electromagnetism, and the strong and weak nuclear forces. It provides a framework where these forces can be described by the vibratio

Because `res` is just another `AIMessage` object, we can append it to `messages`, add another `HumanMessage`, and generate the next response in the conversation.

In [7]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="Why do physicists believe it can produce a 'unified theory'?"
)
# add to messages
messages.append(prompt)

# send to chat-gpt
res = chat(messages)

print(res.content)

Physicists believe that string theory has the potential to produce a unified theory because it incorporates all the fundamental forces of nature within a single framework. Here are a few reasons why string theory is considered a candidate for a unified theory:

1. Gravity and Quantum Mechanics: One of the biggest challenges in physics is reconciling gravity, described by Einstein's theory of general relativity, with quantum mechanics, which governs the behavior of particles at the smallest scales. String theory naturally includes gravity and is inherently a quantum theory, providing a potential framework for unifying these two fundamental theories.

2. Consistency and Renormalizability: String theory resolves some of the mathematical inconsistencies that arise when trying to combine quantum mechanics with general relativity. It is a mathematically consistent theory that avoids the infinities encountered in other attempts at unification, such as quantum field theory.

3. Gauge Symmetry:

# New Prompt Templates

Alongside what we've seen so far there are also three new prompt templates that we can use. Those are the `SystemMessagePromptTemplate`, `AIMessagePromptTemplate`, and `HumanMessagePromptTemplate`.

These are simply an extension of Langchain's prompt templates that modify the returning "prompt" to be a `SystemMessage`, `AIMessage`, or `HumanMessage` object respectively.

For now, there are not a huge number of use-cases for these objects. However, if we have something that we'd like to add to our messages, this can be helpful. For example, let's say we'd like our AI responses to always consist of no more than 50 characters.

Using the current OpenAI `gpt-3.5-turbo-0301` model, we might run into issues if passing this instruction in the first system message only.

In [8]:
# chat = ChatOpenAI(
#     openai_api_key=OPENAI_API_KEY,
#     temperature=0,
#     model='gpt-3.5-turbo-0301'
# )

# setup first system message
messages = [
    SystemMessage(content=(
        'You are a helpful assistant. You keep responses to no more than '
        '100 characters long (including whitespace), and sign off every '
        'message with a random name like "Robot McRobot" or "Bot Rob".'
    )),
    HumanMessage(content="Hi AI, how are you? What is quantum physics?")
]

Now make our first completion.

In [9]:
res = chat(messages)

print(f"Length: {len(res.content)}\n{res.content}")

Length: 119
I'm good! Quantum physics is a branch of physics that studies the behavior of matter and energy at the smallest scales.


Okay, seems like our AI assistant isn't so good at following either of our instructions. What if we add these instructions to the `HumanMessage` via a `HumanMessagePromptTemplate`?

In [10]:
from langchain.prompts.chat import HumanMessagePromptTemplate, ChatPromptTemplate

human_template = HumanMessagePromptTemplate.from_template(
    '{input} Can you keep the response to no more than 100 characters '+
    '(including whitespace), and sign off with a random name like "Robot '+
    'McRobot" or "Bot Rob".'
)

# create the human message
chat_prompt = ChatPromptTemplate.from_messages([human_template])
# format with some input
chat_prompt_value = chat_prompt.format_prompt(
    input="Hi AI, how are you? What is quantum physics?"
)
chat_prompt_value

ChatPromptValue(messages=[HumanMessage(content='Hi AI, how are you? What is quantum physics? Can you keep the response to no more than 100 characters (including whitespace), and sign off with a random name like "Robot McRobot" or "Bot Rob".')])

Note that to use `HumanMessagePromptTemplate` as typical a prompt templates with the `.format_prompt` method, we needed to pass it through a `ChatPromptTemplate` object. This is case for all of the new chat-based prompt templates.

Using this we return a `ChatPromptValue` object. This can be formatted into a list or string like so:

In [11]:
chat_prompt_value.to_messages()

[HumanMessage(content='Hi AI, how are you? What is quantum physics? Can you keep the response to no more than 100 characters (including whitespace), and sign off with a random name like "Robot McRobot" or "Bot Rob".')]

In [12]:
chat_prompt_value.to_string()

'Human: Hi AI, how are you? What is quantum physics? Can you keep the response to no more than 100 characters (including whitespace), and sign off with a random name like "Robot McRobot" or "Bot Rob".'

In [13]:
messages = [
    SystemMessage(content=(
        'You are a helpful assistant. You keep responses to no more than '
        '100 characters long (including whitespace), and sign off every '
        'message with a random name like "Robot McRobot" or "Bot Rob".'
    )),
    chat_prompt.format_prompt(
        input="Hi AI, how are you? What is quantum physics?"
    ).to_messages()[0]
]

res = chat(messages)

print(f"Length: {len(res.content)}\n{res.content}")

Length: 98
I'm good! Quantum physics is a branch of physics that studies the behavior of particles. - Bot Rob


This time we get pretty close, we're slightly over the character limit (by 8 characters), and we got a sign off with - `Bot Rob.`

We can also use the prompt templates approach for building an initial system message with a few examples for the chatbot to follow — __few-shot training via examples. Let's see what that looks like__.

In [14]:
from langchain.prompts.chat import (
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate
)

system_template = SystemMessagePromptTemplate.from_template(
    'You are a helpful assistant. You keep responses to no more than '
    '{character_limit} characters long (including whitespace), and sign '
    'off every message with "- {sign_off}'
)
human_template = HumanMessagePromptTemplate.from_template("{input}")
ai_template = AIMessagePromptTemplate.from_template("{response} - {sign_off}")

# create the list of messages
chat_prompt = ChatPromptTemplate.from_messages([
    system_template,
    human_template,
    ai_template
])
# format with required inputs
chat_prompt_value = chat_prompt.format_prompt(
    character_limit="50", sign_off="Robot McRobot",
    input="Hi AI, how are you? What is quantum physics?",
    response="Good! It's physics of small things"
)
chat_prompt_value

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant. You keep responses to no more than 50 characters long (including whitespace), and sign off every message with "- Robot McRobot'), HumanMessage(content='Hi AI, how are you? What is quantum physics?'), AIMessage(content="Good! It's physics of small things - Robot McRobot")])

We extract these as messages and feed them into the chat model alongside our next query, which we'll feed in as usual (without the template).

In [15]:
messages = chat_prompt_value.to_messages()

messages.append(
    HumanMessage(content="How small?")
)

res = chat(messages)

print(f"Length: {len(res.content)}\n{res.content}")

Length: 52
Very small, like atoms and particles - Robot McRobot


Perfect, we seem to get a good response there, let's try a couple more.

In [16]:
# add last response
messages.append(res)

# make new query
messages.append(
    HumanMessage(content="Okay cool, so it is like 'partical physics'?")
)

res = chat(messages)

print(f"Length: {len(res.content)}\n{res.content}")

Length: 68
Yes, quantum physics is a branch of particle physics - Robot McRobot


We went a little over here. We could begin using the previous `HumanMessagePromptTemplate` again like so:

In [17]:
from langchain import PromptTemplate

# this is a faster way of building the prompt via a PromptTemplate
human_template = HumanMessagePromptTemplate.from_template(
    '{input} Answer in less than {character_limit} characters (including whitespace).'
)
# create the human message
human_prompt = ChatPromptTemplate.from_messages([human_template])
# format with some input
human_prompt_value = human_prompt.format_prompt(
    input="Okay cool, so it is like 'partical physics'?",
    character_limit="50"
)
human_prompt_value

ChatPromptValue(messages=[HumanMessage(content="Okay cool, so it is like 'partical physics'? Answer in less than 50 characters (including whitespace).")])

In [18]:
# drop the last message about partical physics so we can rewrite
messages.pop(-1)

HumanMessage(content="Okay cool, so it is like 'partical physics'?")

In [19]:
messages.extend(human_prompt_value.to_messages())
messages

[SystemMessage(content='You are a helpful assistant. You keep responses to no more than 50 characters long (including whitespace), and sign off every message with "- Robot McRobot'),
 HumanMessage(content='Hi AI, how are you? What is quantum physics?'),
 AIMessage(content="Good! It's physics of small things - Robot McRobot"),
 HumanMessage(content='How small?'),
 AIMessage(content='Very small, like atoms and particles - Robot McRobot'),
 HumanMessage(content="Okay cool, so it is like 'partical physics'? Answer in less than 50 characters (including whitespace).")]

In [20]:
#Now process:

res = chat(messages)

print(f"Length: {len(res.content)}\n{res.content}")

Length: 48
Yes, similar to particle physics - Robot McRobot


There we go, a good answer again!

Now, it's arguable as to whether all of the above is better than simple `f-strings` like:

In [21]:
_input = "Okay cool, so it is like 'partical physics'?"
character_limit = 50

human_message = HumanMessage(content=(
    f"{_input} Answer in less than {character_limit} characters "
    "(including whitespace)."
))

human_message

HumanMessage(content="Okay cool, so it is like 'partical physics'? Answer in less than 50 characters (including whitespace).")

In this example, the above is far simpler. So we wouldn't necessarily recommend using prompt templates over f-strings in all scenarios. But, if you do find yourself in a scenario where they become more useful — you now know how to use them.

To finish off, let's see how the rest of the completion process can be done using the `f-string` formatted message `human_message`:

In [22]:
# drop the last message about partical physics so we can rewrite
messages.pop(-1)

# add f-string formatted message
messages.append(human_message)
messages

[SystemMessage(content='You are a helpful assistant. You keep responses to no more than 50 characters long (including whitespace), and sign off every message with "- Robot McRobot'),
 HumanMessage(content='Hi AI, how are you? What is quantum physics?'),
 AIMessage(content="Good! It's physics of small things - Robot McRobot"),
 HumanMessage(content='How small?'),
 AIMessage(content='Very small, like atoms and particles - Robot McRobot'),
 HumanMessage(content="Okay cool, so it is like 'partical physics'? Answer in less than 50 characters (including whitespace).")]

In [23]:
res = chat(messages)

print(f"Length: {len(res.content)}\n{res.content}")

Length: 48
Yes, similar to particle physics - Robot McRobot
